In [1]:
import pandas as pd
import numpy as np
import time,datetime

In [2]:
from sklearn import preprocessing
from sklearn.model_selection import KFold, RepeatedKFold
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split

/home/zhangyier/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
def datetime2SecondsFrom1970(timeDateStr:str):
    time1=datetime.datetime.strptime(timeDateStr,"%Y-%m-%d %H:%M:%S")
    secondsFrom1970=time.mktime(time1.timetuple())
    return secondsFrom1970

def seconds2Datetime(seconds_from_1970):
    timeArray = time.localtime(seconds_from_1970)#1970秒数
    otherStyleTime = time.strftime("%Y-%m-%d %H:%M:%S", timeArray)
    datetime1=datetime.datetime.strptime(otherStyleTime, "%Y-%m-%d %H:%M:%S")
    return str(datetime1)
def datetime2SecondsFrom1970_s(timeDateStr:str):
    time1=datetime.datetime.strptime(timeDateStr,"%Y%m%d%H%M%S")
    secondsFrom1970=time.mktime(time1.timetuple())
    return int(secondsFrom1970)
def smape(valid_,label_valid_array):
    temp_differ_abs=abs(valid_-label_valid_array)
    temp_sum_mod_2=(valid_+label_valid_array)/2
    print(40*(1-(sum(temp_differ_abs/temp_sum_mod_2)/label_valid_array.shape[0]/2)))
    return sum(temp_differ_abs/temp_sum_mod_2)/label_valid_array.shape[0]

In [4]:
ad_sample_test=pd.read_csv('../Data/B_preliminary_generate/Test/ad_sample_feature1phase_test.csv')

ad_sample_test['have_create_seconds']=0

ad_sample_test['have_create_seconds'][
    ad_sample_test['create_time']<=datetime2SecondsFrom1970_s('20190319235959')]=datetime2SecondsFrom1970_s('20190320235959')-ad_sample_test['create_time'][ad_sample_test['create_time']
                                                                                                                                                 <=datetime2SecondsFrom1970_s('20190319235959')]
#预测3月20日周三

ad_sample_test['have_create_seconds'][(ad_sample_test['create_time']>datetime2SecondsFrom1970_s('20190319235959'))&
(ad_sample_test['create_time']<=datetime2SecondsFrom1970_s('20190320235959'))]=datetime2SecondsFrom1970_s('20190321235959')-ad_sample_test['create_time'][
(ad_sample_test['create_time']>datetime2SecondsFrom1970_s('20190319235959'))&
(ad_sample_test['create_time']<=datetime2SecondsFrom1970_s('20190320235959'))]
#3月21日周四

ad_sample_test['have_create_seconds'][(ad_sample_test['create_time']>datetime2SecondsFrom1970_s('20190320235959'))&
(ad_sample_test['create_time']<=datetime2SecondsFrom1970_s('20190321235959'))]=datetime2SecondsFrom1970_s('20190322235959')-ad_sample_test['create_time'][
(ad_sample_test['create_time']>datetime2SecondsFrom1970_s('20190320235959'))&
(ad_sample_test['create_time']<=datetime2SecondsFrom1970_s('20190321235959'))]
#3月22日周五

ad_sample_test['have_create_seconds'][(ad_sample_test['create_time']>datetime2SecondsFrom1970_s('20190321235959'))&
(ad_sample_test['create_time']<=datetime2SecondsFrom1970_s('20190322235959'))]=datetime2SecondsFrom1970_s('20190323235959')-ad_sample_test['create_time'][
(ad_sample_test['create_time']>datetime2SecondsFrom1970_s('20190321235959'))&
(ad_sample_test['create_time']<=datetime2SecondsFrom1970_s('20190322235959'))]
#3月23日周六

ad_sample_test['have_create_seconds'][(ad_sample_test['create_time']>datetime2SecondsFrom1970_s('20190322235959'))&
(ad_sample_test['create_time']<=datetime2SecondsFrom1970_s('20190323235959'))]=datetime2SecondsFrom1970_s('20190324235959')-ad_sample_test['create_time'][
(ad_sample_test['create_time']>datetime2SecondsFrom1970_s('20190322235959'))&
(ad_sample_test['create_time']<=datetime2SecondsFrom1970_s('20190323235959'))]
#3月24日周日

ad_sample_test['have_create_seconds'][(ad_sample_test['create_time']>datetime2SecondsFrom1970_s('20190323235959'))&
(ad_sample_test['create_time']<=datetime2SecondsFrom1970_s('20190324235959'))]=datetime2SecondsFrom1970_s('20190325235959')-ad_sample_test['create_time'][
(ad_sample_test['create_time']>datetime2SecondsFrom1970_s('20190323235959'))&
(ad_sample_test['create_time']<=datetime2SecondsFrom1970_s('20190324235959'))]
#3月25日周一


/home/zhangyier/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/zhangyier/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [5]:
ad_static_feature=pd.read_csv("../Data/A_preliminary/testA/ad_static_feature.out",header=None,sep='\t',
                             names=['ad_id','create_time','ad_account_id','commodity_id','commodity_type','ad_trades_id','ad_size',
                             ])

/home/zhangyier/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
def train_set_ready(train_set_name_list,ad_static_feature):
    train_set_3_19=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_0319_feature1phase_train.csv')
    valid_set_from_3_19=train_set_3_19[:2000]#设定验证集
    train_set_3_19=train_set_3_19[2000:]
    train_set=train_set_3_19
    for day in train_set_name_list:
        print("date",day)
        if day =='0319':
            continue
        train_set_date=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_'+day+'_feature1phase_train.csv')
        train_set=pd.concat([train_set,train_set_date],axis=0)
        train_set['ad_trades_id']=train_set['ad_trades_id'].apply(lambda x: int(str(x).split(',')[0]))
        #3月18,19日两天有275条样本交易行业超过一个，但是test_sample中不存在这样的样本，直接取第一个
    train_set.drop(columns='create_time',inplace=True)
    valid_set_from_3_19.drop(columns='create_time',inplace=True)
    print('train\n',train_set.info())
    train_set=train_set.merge(ad_static_feature[['ad_id','create_time']],on='ad_id',how='inner')
    valid_set_from_3_19=valid_set_from_3_19.merge(ad_static_feature[['ad_id','create_time']],on='ad_id',how='inner')
    train_set['have_create_seconds']=train_set['end_time']-train_set['create_time']
    valid_set_from_3_19['have_create_seconds']=valid_set_from_3_19['end_time']-valid_set_from_3_19['create_time']
    print('valid\n',valid_set_from_3_19.info())
    
    print("drop duplicates:\n",train_set.shape[0])
    train_set.drop_duplicates(inplace=True)
    print(train_set.shape[0])#去重
    train_set=train_set[(train_set['hold_time']>10800)]#持续时间需要超过三个小时的样本
    # train_set=train_set[(train_set['audience_orientation_nums']>0)]#定向人群数量为0的是有问题的
    train_set.reset_index(inplace=True)
    
    return train_set,valid_set_from_3_19

In [7]:
columns_list=[
    # 'ad_id','audience_targeting','end_time','ad_account_id','commodity_id','commodity_type','ad_trades_id','ad_size','hold_time',
    #  'create_time','sample_id', 'exposure_ad_id',
     'bid','audience_orientation_nums','0000_0030','0030_0100','0100_0130','0130_0200','0200_0230','0230_0300','0300_0330',
    '0330_0400','0400_0430','0430_0500','0500_0530','0530_0600','0600_0630','0630_0700','0700_0730','0730_0800','0800_0830','0830_0900',
    '0900_0930','0930_1000','1000_1030','1030_1100','1100_1130','1130_1200','1200_1230','1230_1300','1300_1330','1330_1400','1400_1430','1430_1500',
    '1500_1530','1530_1600','1600_1630','1630_1700','1700_1730','1730_1800','1800_1830','1830_1900','1900_1930','1930_2000','2000_2030','2030_2100',
    '2100_2130','2130_2200','2200_2230','2230_2300','2300_2330','2330_2400',
    'ad_id_history_exposure_times','ad_id_bid_mean','ad_id_bid_var','ad_id_pctr_mean','ad_id_pctr_var','ad_id_quality_ecpm_mean',
    'ad_id_quality_ecpm_var','ad_id_total_ecpm_mean','ad_id_total_ecpm_var', 
    'account_id_history_exposure_times','account_id_bid_mean','account_id_bid_var','account_id_pctr_mean','account_id_pctr_var',
    'account_id_quality_ecpm_mean','account_id_quality_ecpm_var','account_id_total_ecpm_mean','account_id_total_ecpm_var',
    'commodity_id_history_exposure_times','commodity_id_bid_mean','commodity_id_bid_var','commodity_id_pctr_mean','commodity_id_pctr_var',
    'commodity_id_quality_ecpm_mean','commodity_id_quality_ecpm_var','commodity_id_total_ecpm_mean','commodity_id_total_ecpm_var',
    'commodity_type_history_exposure_times','commodity_type_bid_mean','commodity_type_bid_var','commodity_type_pctr_mean',
    'commodity_type_pctr_var','commodity_type_quality_ecpm_mean','commodity_type_quality_ecpm_var','commodity_type_total_ecpm_mean',
    'commodity_type_total_ecpm_var',
    'trades_id_history_exposure_times','trades_id_bid_mean','trades_id_bid_var','trades_id_pctr_mean','trades_id_pctr_var',
    'trades_id_quality_ecpm_mean','trades_id_quality_ecpm_var','trades_id_total_ecpm_mean','trades_id_total_ecpm_var',
    'week_day','have_create_seconds']
def feture_fixed(columns_list,train_set,ad_sample_test,valid_set_from_3_19):
    print('feature fixed')
    label=train_set['exposure_times']
    train=train_set[columns_list]
    test=ad_sample_test[columns_list]
    label_validation=valid_set_from_3_19['exposure_times']
    validation=valid_set_from_3_19[columns_list]
    
    train_array=train.values
    label_array=label.values
    test_array=test.values
    label_valid_array=label_validation.values
    validtion_array=validation.values
    return train_array,label_array,test_array,label_valid_array,validtion_array

In [8]:
def xgb_train(train_array,label_array,test_array,label_valid_array,validtion_array):
    xgb_params = {'eta': 0.005, 'max_depth': 10, 'subsample': 0.8, 'colsample_bytree': 0.8,
              'objective': 'reg:linear', 'eval_metric': 'rmse', 'silent': True, 'nthread': 10}
    folds = KFold(n_splits=5, shuffle=True, random_state=2019)
    oof_xgb = np.zeros(len(train_array))
    predictions_xgb = np.zeros(len(test_array))
    valid_xgb=np.zeros(len(label_valid_array))
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_array, label_array)):
        print("fold n°{}".format(fold_ + 1))
        trn_data = xgb.DMatrix(train_array[trn_idx], label_array[trn_idx])
        val_data = xgb.DMatrix(train_array[val_idx], label_array[val_idx])

        watchlist = [(trn_data, 'train'), (val_data, 'valid_data')]
        clf = xgb.train(dtrain=trn_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200,
                        verbose_eval=100, params=xgb_params)
        oof_xgb[val_idx] = clf.predict(xgb.DMatrix(train_array[val_idx]), ntree_limit=clf.best_ntree_limit)
        predictions_xgb += clf.predict(xgb.DMatrix(test_array), ntree_limit=clf.best_ntree_limit) / folds.n_splits
        valid_xgb += clf.predict(xgb.DMatrix(validtion_array), ntree_limit=clf.best_ntree_limit) / folds.n_splits

    print("CV score: {:<8.8f}".format(mean_squared_error(oof_xgb, label_array)))
    print("cv smape:",smape(oof_xgb, label_array))
    print('------------------------------------------------------------------------------------------------------------------')
    print('validation:',mean_squared_error(valid_xgb, label_valid_array))
    print(smape(valid_xgb,label_valid_array))
    print('------------------------------------------------------------------------------------------------------------------')

In [9]:
def offline_xgb_test(train_set_name_list,ad_static_feature,ad_sample_test,columns_list):
    train_set,valid_set_from_3_19=train_set_ready(train_set_name_list,ad_static_feature)
    train_array,label_array,test_array,label_valid_array,validtion_array=feture_fixed(columns_list,train_set,ad_sample_test,valid_set_from_3_19)
    xgb_train(train_array,label_array,test_array,label_valid_array,validtion_array)

In [11]:
train_set_name_all_list=['0319','0318','0317','0316','0315','0314','0313','0312','0311','0310','0309','0308','0307','0306','0305','0304',
                         '0303','0302','0301','0228','0227','0226','0225','0224']
for index in range(23,len(train_set_name_all_list)+1):
    train_set_name_list=train_set_name_all_list[:index]
    offline_xgb_test(train_set_name_list,ad_static_feature,ad_sample_test,columns_list)

date 0319
date 0318
date 0317
date 0316
date 0315
date 0314
date 0313
date 0312
date 0311
date 0310
date 0309
date 0308
date 0307
date 0306
date 0305
date 0304
date 0303
date 0302
date 0301
date 0228
date 0227
date 0226
date 0225
<class 'pandas.core.frame.DataFrame'>
Int64Index: 340517 entries, 2000 to 12830
Columns: 108 entries, ad_id to week_day
dtypes: float64(46), int64(61), object(1)
memory usage: 283.2+ MB
train
 None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 1999
Columns: 110 entries, ad_id to have_create_seconds
dtypes: float64(46), int64(63), object(1)
memory usage: 1.7+ MB
valid
 None
drop duplicates:
 340517
340517
feature fixed
fold n°1
[0]	train-rmse:95.1598	valid_data-rmse:80.4291
Multiple eval metrics have been passed: 'valid_data-rmse' will be used for early stopping.

Will train until valid_data-rmse hasn't improved in 200 rounds.
[100]	train-rmse:68.075	valid_data-rmse:64.8499
[200]	train-rmse:50.1819	valid_data-rmse:59.3733
[300]	train-rmse

[1600]	train-rmse:10.2949	valid_data-rmse:43.4599
[1700]	train-rmse:10.113	valid_data-rmse:43.438
[1800]	train-rmse:9.96945	valid_data-rmse:43.4274
[1900]	train-rmse:9.82131	valid_data-rmse:43.4166
[2000]	train-rmse:9.69457	valid_data-rmse:43.4098
[2100]	train-rmse:9.5769	valid_data-rmse:43.414
[2200]	train-rmse:9.45231	valid_data-rmse:43.4134
Stopping. Best iteration:
[2051]	train-rmse:9.63459	valid_data-rmse:43.4057

fold n°2
[0]	train-rmse:93.7376	valid_data-rmse:86.8549
Multiple eval metrics have been passed: 'valid_data-rmse' will be used for early stopping.

Will train until valid_data-rmse hasn't improved in 200 rounds.
[100]	train-rmse:67.3429	valid_data-rmse:73.7688
[200]	train-rmse:49.7094	valid_data-rmse:67.2978
[300]	train-rmse:38.1409	valid_data-rmse:64.05
[400]	train-rmse:30.1384	valid_data-rmse:62.7948
[500]	train-rmse:24.6574	valid_data-rmse:62.2702
[600]	train-rmse:20.8914	valid_data-rmse:62.1457
[700]	train-rmse:18.0729	valid_data-rmse:62.3194
[800]	train-rmse:16.0861

In [25]:
train_set_name_list=['0319','0318','0317','0316','0315','0314','0313','0312','0311','0310','0309','0308','0307','0306','0305','0304','0303','0302','0301','0228','0227','0226']
offline_xgb_test(train_set_name_list,ad_static_feature,ad_sample_test,columns_list)

date 0319
date 0318
date 0317
date 0316
date 0315
date 0314
date 0313
date 0312
date 0311
date 0310
date 0309
date 0308
date 0307
date 0306
date 0305
date 0304
date 0303
date 0302
date 0301
date 0228
date 0227
date 0226
<class 'pandas.core.frame.DataFrame'>
Int64Index: 327686 entries, 2000 to 13062
Columns: 108 entries, ad_id to week_day
dtypes: float64(46), int64(61), object(1)
memory usage: 272.5+ MB
train
 None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 1999
Columns: 110 entries, ad_id to have_create_seconds
dtypes: float64(46), int64(63), object(1)
memory usage: 1.7+ MB
valid
 None
drop duplicates:
 327686
327686
feature fixed
fold n°1
[0]	train-rmse:83.0213	valid_data-rmse:111.483
Multiple eval metrics have been passed: 'valid_data-rmse' will be used for early stopping.

Will train until valid_data-rmse hasn't improved in 200 rounds.
[100]	train-rmse:59.8161	valid_data-rmse:94.7949
[200]	train-rmse:44.5623	valid_data-rmse:85.9082
[300]	train-rmse:34.0558	